In [63]:
import pandas as pd
import numpy as np
import copy
from typing import List

import xgboost as xgb
from tqdm import tqdm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import balanced_accuracy_score, cohen_kappa_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold

import warnings
warnings.filterwarnings("ignore")

In [64]:
d = pd.read_csv("results/attacks_diagnoses.csv")
d = d[(d["dataset"] != "mfeat-morphological") & (d["attack"] != "lpf")]

In [65]:
def multiclass_false_negative_rate(y_true: List[str], y_pred: List[str]) -> float:
    """
    Calculates the average False Negative Rate (FNR) across all classes in a
    multiclass classification.

    The False Negative Rate (FNR) for a class is the proportion of actual
    positive cases for that class that were incorrectly identified as negative.
    FNR = False Negatives / (False Negatives + True Positives)

    This function computes the FNR for each class and then returns the average
    of these values.

    Args:
        y_true (List[str]): A list of true labels.
        y_pred (List[str]): A list of predicted labels.

    Returns:
        float: The average False Negative Rate across all classes.
               Returns 0.0 if there are no classes with true positives.
    """
    if len(y_true) != len(y_pred):
        raise ValueError("Input arrays y_true and y_pred must have the same length.")

    unique_classes = np.unique(y_true)
    fnr_values = []

    for c in unique_classes:
        # Identify instances where the true label is the current class 'c'
        actual_positives = np.array(y_true) == c

        # Find true positives (y_true == c AND y_pred == c)
        true_positives = np.sum((np.array(y_pred) == c) & actual_positives)

        # Find false negatives (y_true == c AND y_pred != c)
        false_negatives = np.sum((np.array(y_pred) != c) & actual_positives)

        # Calculate FNR for the current class
        total_positives = true_positives + false_negatives
        if total_positives == 0:
            fnr = 0.0
        else:
            fnr = false_negatives / total_positives

        fnr_values.append(fnr)

    # Calculate the average FNR
    if not fnr_values:
        return 0.0
    return np.mean(fnr_values)

In [66]:
def q0(x: float) -> float:
    return x.quantile(0)


def q25(x: float) -> float:
    return x.quantile(0.25)


def q50(x: float) -> float:
    return x.quantile(0.5)


def q75(x: float) -> float:
    return x.quantile(0.75)


def q1(x: float) -> float:
    return x.quantile(1)


def minmax(x: float) -> float:
    return x.max() - x.min()

def false_negative_rate(y_true, y_pred):
    if confusion_matrix(y_true, y_pred).shape == (2, 2):
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
        return fn / (fn + tp)
    else:
        return np.nan

attrs_con = d
cols_to_drop = ["approx", "target", "pred", "error", "name",
                    "overall_mean_target", "scores",
                    "mean_target_in_neighborhood",
                    "mean_approx_in_neighborhood",
                    "neighborhood_size_div_model_avg",
                    "neighborhood_size_pct",
                    "r_centered_entropy",
                "entropy",
                    "logk_r_centered_entropy",
                    "bacc_test"]

attrs_cols = attrs_con.drop(columns=cols_to_drop)
attrs_agg = attrs_cols \
    .groupby(["dataset", "model", "attack", 
              "n_test", "n_classes"]) \
    .agg(['mean', q0, q25, q50, q75, q1, minmax])
attrs_agg_correct_cols = copy.deepcopy(attrs_agg)
attrs_agg_correct_cols.columns = list(attrs_agg_correct_cols.columns.map('_'.join))
attrs_agg_correct_cols = attrs_agg_correct_cols.reset_index()

In [67]:
d_nn = pd.read_csv("results/attacks_diagnoses_nn.csv")

attrs_cols = d_nn.drop(columns=cols_to_drop)
attrs_agg = attrs_cols \
    .groupby(["dataset", "model", "attack", 
              "n_test", "n_classes"]) \
    .agg(['mean', q0, q25, q50, q75, q1, minmax])
attrs_agg_correct_cols_nn = copy.deepcopy(attrs_agg)
attrs_agg_correct_cols_nn.columns = list(attrs_agg_correct_cols_nn.columns.map('_'.join))
attrs_agg_correct_cols_nn = attrs_agg_correct_cols_nn.reset_index()

In [68]:
attrs_agg_correct_cols = pd.concat([attrs_agg_correct_cols, attrs_agg_correct_cols_nn])

In [69]:
np.unique(attrs_agg_correct_cols["attack"], return_counts=True)

(array(['bim', 'fgm', 'hsj', 'noise', 'org', 'per', 'pgd', 'zoo'],
       dtype=object),
 array([22, 22, 64, 22, 88, 66, 22, 66]))

In [70]:
data_counts = attrs_agg_correct_cols[["dataset", "model", "attack"]].groupby(["dataset", "model"]).size().reset_index(name='counts')
data_counts

,dataset,model,counts
0,Bioresponse,lin,4
1,Bioresponse,nn,5
2,Bioresponse,svm,4
3,Bioresponse,xgb,4
4,churn,lin,4
...,...,...,...
83,wdbc,xgb,4
84,wilt,lin,4
85,wilt,nn,5
86,wilt,svm,4


In [71]:
data_counts_model_type = attrs_agg_correct_cols[["dataset", "model", "attack"]].groupby(["dataset", "attack"]).size().reset_index(name='counts')
data_counts_model_type

,dataset,attack,counts
0,Bioresponse,bim,1
1,Bioresponse,fgm,1
2,Bioresponse,hsj,3
3,Bioresponse,noise,1
4,Bioresponse,org,4
...,...,...,...
171,wilt,noise,1
172,wilt,org,4
173,wilt,per,3
174,wilt,pgd,1


In [72]:
le = LabelEncoder()
x_train = attrs_agg_correct_cols.drop(columns=["dataset", "model", "attack"])
y_train = attrs_agg_correct_cols["attack"]
y_train_enc = le.fit_transform(y_train)
model = RandomForestClassifier(random_state=123)
model.fit(x_train, y_train_enc)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [73]:
fi_dict = {'var': x_train.columns, 'fi': model.feature_importances_}
fi_df = pd.DataFrame(fi_dict)
fi_df["fi_rank"] = fi_df["fi"].rank(ascending=False)

In [74]:
attrs_agg_correct_cols.to_csv("results/attr_attacks_type_agr_nn_bacc.csv", index=False)

In [75]:
def create_hyperparams_grid(model, param_grid, exp_function, crit="kappa"):

    quality_measure = []
    params_list = []
    results_table = []
    results_fi = []

    for param1 in param_grid[list(param_grid.keys())[0]]:
        for param2 in param_grid[list(param_grid.keys())[1]]:
            for param3 in param_grid[list(param_grid.keys())[2]]:
                params = {list(param_grid.keys())[0]: param1,
                          list(param_grid.keys())[1]: param2,
                          list(param_grid.keys())[2]: param3}

                model.set_params(**params)
                summary, fi = exp_function(model=model)
                results_table.append(summary)
                results_fi.append(fi)
                params_list.append(params)
                crit_value = np.mean(summary["kappa"])
                quality_measure.append(crit_value)
                print(f"{crit} = {crit_value}, params: {params}")

    best_params_idx = np.argmax(quality_measure)
    best_params = params_list[best_params_idx]

    print(f"Params: {best_params} give best {crit} equal {quality_measure[best_params_idx]}")

    return results_table[best_params_idx], results_fi[best_params_idx]

# Leave-one-data-set-out

In [76]:
def leave_dataset_out(model=xgb.XGBClassifier(random_state=123), params=None):

    datasets = np.unique(attrs_agg_correct_cols["dataset"])
    kappa_list = []
    bacc_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    fnr_list = []
    fi_all = pd.DataFrame()

    for dataset in tqdm(datasets):

        train = attrs_agg_correct_cols[attrs_agg_correct_cols["dataset"] != dataset]
        test = attrs_agg_correct_cols[attrs_agg_correct_cols["dataset"] == dataset]

        x_train = train.drop(columns=["dataset", "model", "attack"])
        y_train = train["attack"]

        x_test = test.drop(columns=["dataset", "model", "attack"])
        y_test = test["attack"]

        le = LabelEncoder()
        y_train_enc = le.fit_transform(y_train)
        y_test_enc = le.transform(y_test)

        if params is not None:
            model.set_params(**params)
        model.fit(x_train, y_train_enc)

        fi_dict = {'var': x_train.columns, 'fi': model.feature_importances_}
        fi_df = pd.DataFrame(fi_dict)
        fi_df["fi_rank"] = fi_df["fi"].rank(ascending=False)
        fi_df["dataset"] = dataset
        fi_all = pd.concat([fi_all, fi_df])

        preds = model.predict(x_test)
        bacc_list.append(balanced_accuracy_score(y_test_enc, preds))
        kappa_list.append(cohen_kappa_score(y_test_enc, preds))
        precision_list.append(precision_score(y_test_enc, preds, average='weighted'))
        recall_list.append(recall_score(y_test_enc, preds, average='weighted'))
        f1_list.append(f1_score(y_test_enc, preds, average='weighted'))
        fnr_list.append(multiclass_false_negative_rate(y_test_enc, preds))

    results_dict = {'dataset': datasets, 'bacc': bacc_list, 'kappa': kappa_list, 'precision': precision_list,
                    'recall': recall_list, 'f1': f1_list, 'fnr': fnr_list}

    results_df = pd.DataFrame(results_dict)
    return results_df, fi_all

## XGBoost

### Default parameters

In [77]:
one_dataset_out_xgb, one_dataset_out_fi_xgb = leave_dataset_out()
one_dataset_out_xgb

100%|██████████| 22/22 [00:26<00:00,  1.20s/it]


,dataset,bacc,kappa,precision,recall,f1,fnr
0,Bioresponse,0.687500,0.442623,0.691176,0.529412,0.544563,0.312500
1,churn,0.541667,0.508264,0.470588,0.588235,0.501961,0.458333
2,cmc,0.875000,0.929752,0.911765,0.941176,0.921569,0.125000
3,cnae-9,0.604167,0.447154,0.521008,0.529412,0.482353,0.395833
4,dna,0.583333,0.576763,0.558824,0.647059,0.580719,0.416667
5,har,0.666667,0.573222,0.693277,0.647059,0.616221,0.333333
6,madelon,0.500000,0.221239,0.208333,0.312500,0.223214,0.500000
7,mfeat-factors,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
8,mfeat-fourier,0.583333,0.578512,0.588235,0.647059,0.592157,0.416667
9,mfeat-karhunen,0.875000,0.929752,0.911765,0.941176,0.921569,0.125000


### Hyperparameters tuning

In [78]:
retrain = False

if retrain:
    model=xgb.XGBClassifier(random_state=123)

    param_grid = {
        'max_depth': [6, 9, 12],
        'learning_rate': [0.1, 0.3, 0.5],
        'n_estimators': [100, 200, 500]
    }

    one_dataset_out_xgb, one_dataset_out_fi_xgb = create_hyperparams_grid(model=model, param_grid=param_grid, exp_function=leave_dataset_out, crit="kappa")
    one_dataset_out_xgb
else:
    # Params: {'max_depth': 6, 'learning_rate': 0.5, 'n_estimators': 100} give best kappa equal 0.6037758057986143
    params = {'max_depth': 6, 'learning_rate': 0.5, 'n_estimators': 100, 'random_state': 123}
    one_dataset_out_xgb, one_dataset_out_fi_xgb = leave_dataset_out(model=xgb.XGBClassifier(random_state=123), params=params)

100%|██████████| 22/22 [00:22<00:00,  1.00s/it]


In [79]:
one_dataset_out_xgb[["bacc", "kappa"]].agg(['mean', 'std'])

,bacc,kappa
mean,0.648201,0.603776
std,0.247151,0.276664


In [80]:
def format_mean_std(dataset):   
    metrics = ["bacc", "precision", "recall", "f1", "fnr"] 
    stats = dataset[metrics].agg(['mean', 'std']).round(2)
    formatted = {col: f"{stats.loc['mean',col]:.2f} ({stats.loc['std',col]:.2f})" for col in metrics}
    df = pd.DataFrame([formatted])
    print(df.to_latex(index=False, header=True, escape=False))

format_mean_std(one_dataset_out_xgb)

\begin{tabular}{lllll}
\toprule
bacc & precision & recall & f1 & fnr \\
\midrule
0.65 (0.25) & 0.64 (0.25) & 0.67 (0.24) & 0.63 (0.25) & 0.35 (0.25) \\
\bottomrule
\end{tabular}



In [81]:
fi_agg = one_dataset_out_fi_xgb[["var", "fi_rank"]].groupby(["var"]).agg(['mean'])
fi_agg.columns = fi_agg.columns.to_flat_index()
fi_agg.columns = list(fi_agg.columns.map('_'.join))
fi_agg.nsmallest(n=20, columns=["fi_rank_mean"])

,fi_rank_mean
var,
target_approx_consistency_in_neighborhood_mean,2.000000
target_targets_consistency_in_neighborhood_q50,6.000000
pred_targets_consistency_in_neighborhood_q25,6.636364
uncertainty_q50,6.727273
uncertainty_q25,8.727273
uncertainty_q0,9.454545
uncertainty_q75,10.772727
uncertainty_mean,11.000000
target_targets_consistency_in_neighborhood_mean,11.863636


## Random forest

### Default parameters

In [82]:
one_dataset_out_rf, one_dataset_out_fi_rf = leave_dataset_out(model=RandomForestClassifier(random_state=123))
one_dataset_out_rf

100%|██████████| 22/22 [00:14<00:00,  1.47it/s]


,dataset,bacc,kappa,precision,recall,f1,fnr
0,Bioresponse,0.458333,0.170732,0.176471,0.294118,0.203922,0.541667
1,churn,0.500000,0.438017,0.369748,0.529412,0.419608,0.500000
2,cmc,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
3,cnae-9,0.541667,0.510288,0.443137,0.588235,0.496265,0.458333
4,dna,0.666667,0.717842,0.747059,0.764706,0.742484,0.333333
5,har,0.708333,0.500000,0.529412,0.588235,0.533333,0.291667
6,madelon,0.500000,0.228070,0.177083,0.312500,0.202381,0.500000
7,mfeat-factors,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
8,mfeat-fourier,0.750000,0.719008,0.735294,0.764706,0.740056,0.250000
9,mfeat-karhunen,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000


### Hyperparameters tuning

In [83]:
retrain = False

if retrain:

    model=RandomForestClassifier(random_state=123)

    param_grid = {
        'max_depth': [50, 80, 110],
        'min_samples_split': [2, 5, 8],
        'n_estimators': [100, 200, 500]
    }

    one_dataset_out_rf, one_dataset_out_fi_rf = create_hyperparams_grid(model=model, param_grid=param_grid, exp_function=leave_dataset_out, crit="kappa")
else:
    # Params: {'max_depth': 50, 'min_samples_split': 2, 'n_estimators': 200} give best kappa equal 0.5699655896019489
    params = {'max_depth': 50, 'min_samples_split': 2, 'n_estimators': 200, 'random_state': 123}
    one_dataset_out_rf, one_dataset_out_fi_rf = leave_dataset_out(model=RandomForestClassifier(random_state=123), params=params)

100%|██████████| 22/22 [00:33<00:00,  1.51s/it]


In [84]:
one_dataset_out_rf[["bacc", "kappa"]].agg(['mean', 'std'])

,bacc,kappa
mean,0.644886,0.569966
std,0.272676,0.327542


In [85]:
format_mean_std(one_dataset_out_rf)

\begin{tabular}{lllll}
\toprule
bacc & precision & recall & f1 & fnr \\
\midrule
0.64 (0.27) & 0.59 (0.32) & 0.64 (0.28) & 0.59 (0.31) & 0.36 (0.27) \\
\bottomrule
\end{tabular}



In [86]:
fi_agg = one_dataset_out_fi_rf[["var", "fi_rank"]].groupby(["var"]).agg(['mean'])
fi_agg.columns = fi_agg.columns.to_flat_index()
fi_agg.columns = list(fi_agg.columns.map('_'.join))
fi_agg.nsmallest(n=20, columns=["fi_rank_mean"])

,fi_rank_mean
var,
uncertainty_q50,1.954545
uncertainty_q75,2.272727
uncertainty_q25,2.545455
uncertainty_mean,3.227273
uncertainty_minmax,5.454545
target_approx_consistency_in_neighborhood_mean,6.636364
uncertainty_q1,7.272727
target_approx_consistency_in_neighborhood_q50,7.318182
target_approx_consistency_in_neighborhood_q25,8.500000


# Leave-one-model-out

In [87]:
def leave_model_out(model=xgb.XGBClassifier(random_state=123)):

    models = np.unique(attrs_agg_correct_cols["model"])
    kappa_list = []
    bacc_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    fnr_list = []
    fi_all = pd.DataFrame()

    for selected_model in models:

        train = attrs_agg_correct_cols[attrs_agg_correct_cols["model"] != selected_model]
        test = attrs_agg_correct_cols[attrs_agg_correct_cols["model"] == selected_model]

        y = attrs_agg_correct_cols["attack"]

        x_train = train.drop(columns=["dataset", "model", "attack"])
        y_train = train["attack"]

        x_test = test.drop(columns=["dataset", "model", "attack"])
        y_test = test["attack"]

        le = LabelEncoder()
        # le.fit(y)
        y_train_enc = le.fit_transform(y_train)
        # y_test_enc = le.transform(y_test)

        model.fit(x_train, y_train_enc)

        fi_dict = {'var': x_train.columns, 'fi': model.feature_importances_}
        fi_df = pd.DataFrame(fi_dict)
        fi_df["fi_rank"] = fi_df["fi"].rank(ascending=False)
        fi_df["model"] = selected_model
        fi_all = pd.concat([fi_all, fi_df])

        preds = model.predict(x_test)
        bacc_list.append(balanced_accuracy_score(y_test, le.inverse_transform(preds)))
        kappa_list.append(cohen_kappa_score(y_test, le.inverse_transform(preds)))
        precision_list.append(precision_score(y_test, le.inverse_transform(preds), average='weighted'))
        recall_list.append(recall_score(y_test, le.inverse_transform(preds), average='weighted'))
        f1_list.append(f1_score(y_test, le.inverse_transform(preds), average='weighted'))
        fnr_list.append(multiclass_false_negative_rate(y_test, le.inverse_transform(preds)))

    results_dict = {'model': models, 'bacc': bacc_list, 'kappa': kappa_list, 'precision': precision_list,
                    'recall': recall_list, 'f1': f1_list, 'fnr': fnr_list}

    results_df = pd.DataFrame(results_dict)
    return results_df, fi_all

## XGBoost

In [88]:
one_model_out_xgb, one_model_out_fi_xgb = leave_model_out()
one_model_out_xgb

,model,bacc,kappa,precision,recall,f1,fnr
0,lin,0.681818,0.575758,0.692609,0.681818,0.673272,0.318182
1,nn,0.172727,0.128352,0.135714,0.172727,0.152000,0.827273
2,svm,0.681818,0.578947,0.721462,0.681818,0.692272,0.318182
3,xgb,0.504545,0.360028,0.638663,0.511628,0.457349,0.495455


In [89]:
one_model_out_xgb[["bacc", "kappa"]].agg(['mean', 'std'])

,bacc,kappa
mean,0.510227,0.410771
std,0.240018,0.214351


In [90]:
format_mean_std(one_model_out_xgb)

\begin{tabular}{lllll}
\toprule
bacc & precision & recall & f1 & fnr \\
\midrule
0.51 (0.24) & 0.55 (0.28) & 0.51 (0.24) & 0.49 (0.25) & 0.49 (0.24) \\
\bottomrule
\end{tabular}



## Random forest

In [91]:
one_model_out_rf, one_model_out_fi_rf = leave_model_out(model=RandomForestClassifier(random_state=123))
one_model_out_rf

,model,bacc,kappa,precision,recall,f1,fnr
0,lin,0.670455,0.560606,0.671470,0.670455,0.664605,0.329545
1,nn,0.181818,0.144487,0.166667,0.181818,0.173913,0.818182
2,svm,0.670455,0.560606,0.671667,0.670455,0.665330,0.329545
3,xgb,0.457955,0.287977,0.497565,0.465116,0.379278,0.542045


In [92]:
one_model_out_rf[["bacc", "kappa"]].agg(['mean', 'std'])

,bacc,kappa
mean,0.495170,0.388419
std,0.231678,0.207275


In [93]:
format_mean_std(one_model_out_rf)

\begin{tabular}{lllll}
\toprule
bacc & precision & recall & f1 & fnr \\
\midrule
0.50 (0.23) & 0.50 (0.24) & 0.50 (0.23) & 0.47 (0.24) & 0.50 (0.23) \\
\bottomrule
\end{tabular}



# 10-fold cross validation

In [94]:
def cross_validation(model=xgb.XGBClassifier(random_state=123)):

    kappa_list = []
    bacc_list = []
    precision_list = []
    recall_list = []
    f1_list = []
    fnr_list = []
    iter_cv = []
    fi_all = pd.DataFrame()

    le = LabelEncoder()
    x = attrs_agg_correct_cols.drop(columns=["dataset", "model", "attack"])
    y = attrs_agg_correct_cols["attack"]
    y_enc = le.fit_transform(y)

    skf = StratifiedKFold(n_splits=10)
    cv = 0
    for train, test in skf.split(x, y_enc):

        x_train = x.iloc[train]
        y_train = y_enc[train]

        x_test = x.iloc[test]
        y_test = y_enc[test]

        model.fit(x_train, y_train)

        fi_dict = {'var': x_train.columns, 'fi': model.feature_importances_}
        fi_df = pd.DataFrame(fi_dict)
        fi_df["fi_rank"] = fi_df["fi"].rank(ascending=False)
        fi_df["cv"] = cv
        fi_all = pd.concat([fi_all, fi_df])

        preds = model.predict(x_test)
        bacc_list.append(balanced_accuracy_score(y_test, preds))
        kappa_list.append(cohen_kappa_score(y_test, preds))
        precision_list.append(precision_score(y_test, preds, average='weighted'))
        recall_list.append(recall_score(y_test, preds, average='weighted'))
        f1_list.append(f1_score(y_test, preds, average='weighted'))
        fnr_list.append(multiclass_false_negative_rate(y_test, preds))
        iter_cv.append(cv)
        cv = cv + 1

    results_dict = {'cv': iter_cv, 'bacc': bacc_list, 'kappa': kappa_list, 'precision': precision_list,
                    'recall': recall_list, 'f1': f1_list, 'fnr': fnr_list}
    results_df = pd.DataFrame(results_dict)
    return results_df, fi_all

## XGBoost

In [95]:
cv_xgb, cv_fi_xgb = cross_validation()
cv_xgb

,cv,bacc,kappa,precision,recall,f1,fnr
0,0,0.729167,0.654260,0.693421,0.710526,0.681269,0.270833
1,1,0.713294,0.656250,0.756067,0.710526,0.709693,0.286706
2,2,0.417411,0.254163,0.352968,0.378378,0.351838,0.582589
3,3,0.687500,0.674296,0.777778,0.729730,0.728272,0.312500
4,4,0.752976,0.774194,0.834685,0.810811,0.812006,0.247024
5,5,0.624008,0.645161,0.729955,0.702703,0.689437,0.375992
6,6,0.542659,0.420366,0.571253,0.513514,0.510773,0.457341
7,7,0.500992,0.445326,0.570528,0.540541,0.519235,0.499008
8,8,0.739087,0.708406,0.792383,0.756757,0.758559,0.260913
9,9,0.226190,0.250880,0.389077,0.378378,0.349910,0.773810


In [96]:
cv_xgb[["bacc", "kappa"]].agg(['mean', 'std'])

,bacc,kappa
mean,0.593328,0.548330
std,0.171987,0.190539


In [97]:
format_mean_std(cv_xgb)

\begin{tabular}{lllll}
\toprule
bacc & precision & recall & f1 & fnr \\
\midrule
0.59 (0.17) & 0.65 (0.17) & 0.62 (0.16) & 0.61 (0.17) & 0.41 (0.17) \\
\bottomrule
\end{tabular}



## Random forest

In [98]:
cv_rf, cv_fi_rf = cross_validation(model=RandomForestClassifier(random_state=123))
cv_rf

,cv,bacc,kappa,precision,recall,f1,fnr
0,0,0.586310,0.525790,0.636842,0.605263,0.597343,0.413690
1,1,0.628968,0.562140,0.591447,0.631579,0.602724,0.371032
2,2,0.381696,0.186456,0.385863,0.324324,0.308401,0.618304
3,3,0.723214,0.739207,0.827252,0.783784,0.782629,0.276786
4,4,0.857143,0.838569,0.886486,0.864865,0.865288,0.142857
5,5,0.661706,0.647008,0.784521,0.702703,0.711384,0.338294
6,6,0.500992,0.418341,0.524324,0.513514,0.490627,0.499008
7,7,0.587302,0.475642,0.608559,0.567568,0.554201,0.412698
8,8,0.752976,0.739437,0.841810,0.783784,0.775115,0.247024
9,9,0.247024,0.245567,0.347297,0.378378,0.324939,0.752976


In [99]:
cv_rf[["bacc", "kappa"]].agg(['mean', 'std'])

,bacc,kappa
mean,0.592733,0.537816
std,0.179884,0.213782


In [100]:
format_mean_std(cv_rf)

\begin{tabular}{lllll}
\toprule
bacc & precision & recall & f1 & fnr \\
\midrule
0.59 (0.18) & 0.64 (0.19) & 0.62 (0.18) & 0.60 (0.19) & 0.41 (0.18) \\
\bottomrule
\end{tabular}



## Save data

In [101]:
def preprocess_output(df, scenario, mod):
    df["scenario"] = scenario
    df["model_class"] = mod

    return df

one_dataset_out_rf = preprocess_output(one_dataset_out_rf, "one-data-set-out", "RF")
one_dataset_out_xgb = preprocess_output(one_dataset_out_xgb, "one-data-set-out", "XGB")
one_model_out_rf = preprocess_output(one_model_out_rf, "one-model-out", "RF")
one_model_out_xgb = preprocess_output(one_model_out_xgb, "one-model-out", "XGB")
cv_rf = preprocess_output(cv_rf, "10-fold cross-validation", "RF")
cv_xgb = preprocess_output(cv_xgb, "10-fold cross-validation", "XGB")

all_bacc = pd.concat([one_dataset_out_rf, one_dataset_out_xgb,
                      one_model_out_rf, one_model_out_xgb,
                      cv_rf, cv_xgb])

In [102]:
all_bacc.to_csv("results/isolation_bacc_without_bacc.csv", index=False)

In [103]:
one_dataset_out_fi_rf = preprocess_output(one_dataset_out_fi_rf, "one-data-set-out", "RF")
one_dataset_out_fi_xgb = preprocess_output(one_dataset_out_fi_xgb, "one-data-set-out", "XGB")
one_model_out_fi_rf = preprocess_output(one_model_out_fi_rf, "one-model-out", "RF")
one_model_out_fi_xgb = preprocess_output(one_model_out_fi_xgb, "one-model-out", "XGB")
cv_fi_rf = preprocess_output(cv_fi_rf, "10-fold cross-validation", "RF")
cv_fi_xgb = preprocess_output(cv_fi_xgb, "10-fold cross-validation", "XGB")

all_fi = pd.concat([one_dataset_out_fi_rf, one_dataset_out_fi_xgb,
                      one_model_out_fi_rf, one_model_out_fi_xgb,
                      cv_fi_rf, cv_fi_xgb])

all_fi.to_csv("results/isolation_fi_without_bacc.csv", index=False)